In [1]:
import numpy as np
import pandas as pd
from tabulate import tabulate
from math import sqrt, exp
from scipy.optimize import minimize

In [2]:
r0 = 0.0008588
dt = 1/12
vol = 0.008007
mu = [0., 0,0]
T =3
a = [0.999804602425927,0.999657067401416]

In [3]:
def r_tree(mu_,r0_,vol,dt):
  T = len(mu_)
  rates_ = np.zeros((T,T))
  for j in range(0,T):
    for i in range(j,T):
      if (i==0) and (j==0):
        rates_[T-1,i] = r0_ 
      elif (i > 0) and (j==0): 
        rates_[T-1,i] = rates_[T-1,i-1] + mu_[i]*dt - vol*sqrt(dt) 
      else:
        rates_[T-1-j,i] = rates_[T-1,i] + 2.*j*vol*sqrt(dt) 
    
  return(rates_)

print(tabulate(r_tree(mu,r0,vol,dt)))

---------  -----------  -----------
0           0            0.00548164
0           0.00317022   0.0008588
0.0008588  -0.00145262  -0.00376404
---------  -----------  -----------


In [4]:
r_tr = r_tree(mu[:T],r0,vol,dt)
def bondf(r_tr,dt):
  T = r_tr.shape[0]
  zcb_tr = np.zeros((T,T))
  for i in range(0,T):
    for j in range(i,T):
      if i == 0:
        zcb_tr[j,T-1-i] = 1*exp(-r_tr[j,T-1-i]*dt)
      else:
        zcb_tr[j,T-1-i] = .5*(zcb_tr[j,T-i] + zcb_tr[j-1,T-i]) \
                            *exp(-r_tr[j,T-1-i]*dt)

  return zcb_tr

print(tabulate(bondf(r_tr,dt)))

--------  --------  --------
0         0         0.999543
0         0.999472  0.999928
0.999785  1.00024   1.00031
--------  --------  --------


In [5]:
def bond_err(mu_,a_,param,m_old_):
  r0,vol,dt = param[0],param[1],param[2]
  T = len(m_old_)+1
  r_tr = r_tree(m_old_.__add__([mu_]),r0,vol,dt)
  b_tr = bondf(r_tr,dt)
  return (a_[T-2] - b_tr[T-1,0])**2 

In [6]:
data_set = pd.read_csv('DF.csv',header = None)
a_bond = np.squeeze(np.array(data_set.values))
T = 2
params = [r0,vol,dt]
m_old = [0.]
for i in range(T):
  temp = minimize(bond_err,m_old[-1], \
                  args=(a_bond,params,m_old),method="L-BFGS-B")
  m_old = m_old.__add__([temp.x[0]])
  r_tree_temp = r_tree(m_old,r0,vol,dt)
  print(m_old)
  print(tabulate(r_tree_temp))
  print(tabulate(bondf(r_tree_temp,dt)))

[0.0, 0.0]
---------  -----------
0           0.00317022
0.0008588  -0.00145262
---------  -----------
--------  --------
0         0.999736
0.999857  1.00012
--------  --------
[0.0, 0.0, 0.0]
---------  -----------  -----------
0           0            0.00548164
0           0.00317022   0.0008588
0.0008588  -0.00145262  -0.00376404
---------  -----------  -----------
--------  --------  --------
0         0         0.999543
0         0.999472  0.999928
0.999785  1.00024   1.00031
--------  --------  --------
